#### Day 19 - B
Check which designs can be made using combinations of different towels.

Count the unqiue combinations of making each design.

1) Build trimmed towel lookup (tl2) and create dictionary mapping towel pieces to larger towels

In [326]:
#Import Libraries and settings
from copy import deepcopy
from itertools import combinations as c

settings = {
    "day": 19,
    "test_data": 0
}

In [327]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    towels = lines[0].split(", ")

    #Convert towels into dictionary grouped by the first character
    tlu = {}
    for towel in towels:
        start = towel[0]
        if start not in tlu.keys():
            tlu[start] = [towel]
        else:
            tlu[start].append(towel)

    designs = lines[2:]

    return towels, tlu, designs

towels, tlu, designs = load_input(settings)

In [328]:
#Attempt to recursively make the design from towels
def match_design(tlu, design):

    #Get the next character in the design
    next_char = design[0]
    remaining_design = len(design)

    #Check there are towels starting with this pattern
    if next_char not in tlu.keys():
        return False

    #Try all possible options
    for towel in tlu[next_char]:
        #If this towel matches the next few characters of the design
        if len(towel) < remaining_design:
            if towel == design[0:len(towel)]:
                res = match_design(tlu, design[len(towel):])
                if res:
                    return towel + res


        #If this towel matches the rest of the design    
        elif len(towel) == remaining_design:
            if towel == design:
                return towel
            
    return False

In [329]:
#Attempt to recursively make the design from towels
def match_design_all(tlu, design):

    #Get the next character in the design
    next_char = design[0]
    remaining_design = len(design)

    #Check there are towels starting with this pattern
    if next_char not in tlu.keys():
        return False

    #Try all possible options
    matches = []
    for towel in tlu[next_char]:
        #If this towel matches the next few characters of the design
        if len(towel) < remaining_design:
            if towel == design[0:len(towel)]:
                res = match_design_all(tlu, design[len(towel):])
                if res:
                    for option in res:
                        matches.append([towel] + option)


        #If this towel matches the rest of the design    
        elif len(towel) == remaining_design:
            if towel == design:
                return [[towel]]
            
    return matches

In [330]:
#Get all towels of a given length
#When exact is false it also returns towels shorter than the specified length
def all_towels_of_len_n(tlu, length, exact=True):
    matches = []

    #Get all towels of the required length
    for key in tlu.keys():
        for towel in tlu[key]:
            if exact:
                if len(towel) == length:
                    matches.append(towel)
            else:
                if len(towel) <= length:
                    matches.append(towel)

    #If exact is False, return towels as a dictionary grouped by length
    if exact == False:
        tlu_p = {}
        for piece in matches:
            start = piece[0]
            if start not in tlu_p.keys():
                tlu_p[start] = [piece]
            else:
                tlu_p[start].append(piece)
        return tlu_p
    else:
        return matches

#Convert a full towel lookup
def rebuild(tlu, max=8, status=True):

    if status:
        print("Rebuilding towel lookup dictionary...")

    #Create a copy to avoid overwriting the original
    tl2 = deepcopy(tlu)
    breakdown = {}
    #Track which non_minimum towels have already been made to prevent duplication
    breakdowns_made = set()

    #For each possible towel length (in ascending order)
    for length in range(2, max+1):
        redundant_towels = []
        #Get all towels of current length
        towel_targets = all_towels_of_len_n(tl2, length, exact=True)
        #Get all towels of a shorter length
        towel_pieces = all_towels_of_len_n(tl2, length-1, exact=False)

        #For each towel or the specified length
        for target in towel_targets:
            #Attempt to make the towel using smaller ones
            combinations = match_design_all(towel_pieces, target)

            if combinations:
                #If the towel can be made from smaller towels, it is redundant
                redundant_towels.append(target)
                #Record ways to make this towel in the breakdown dictionary
                for combo in combinations:
                    #If this combo has not already been made
                    if "".join(combo) not in breakdowns_made:
                        #Add it to both the breakdown dictionary and the set of made breakdowns
                        breakdown[tuple(combo)] = "".join(combo)
                        breakdowns_made.add("".join(combo))

        #Remove redundant towels from tlu
        for key in tl2.keys():
            tl2[key] = list(filter(lambda x: x not in redundant_towels, tl2[key]))
        
        #Print processing status
        if status:
            print("Towels of length", length, "processed.")

    if status:
        print()
    return tl2, breakdown

In [331]:
def get_possible_designs(towel_lookup, designs, status=True):
    solutions = []

    if status:
        print("Processing", len(designs), "designs:")

    #For each design, check if it can be made using the towel lookup dict
    for idx, design in enumerate(designs):

        if status and idx%100 == 0:
            print("Processing design", str(idx)+"...")

        res = match_design_all(towel_lookup, design)
        solutions.append(res)

    return solutions

def trim_impossible_designs(towel_lookup, designs, status=True):
    possible_designs = []

    if status:
        print("Processing", len(designs), "designs:")

    #For each design, check if it can be made using the towel lookup dict
    for idx, design in enumerate(designs):
        if status and idx%100 == 0:
            print("Processing design", str(idx)+"...")

        res = match_design(towel_lookup, design)
        if res:
            possible_designs.append(design)

    return possible_designs

In [332]:
tl2 = deepcopy(tlu)
tl2, breakdown = rebuild(tl2)

Rebuilding towel lookup dictionary...
Towels of length 2 processed.
Towels of length 3 processed.
Towels of length 4 processed.
Towels of length 5 processed.
Towels of length 6 processed.
Towels of length 7 processed.
Towels of length 8 processed.



2) Remove all impossible designs from the list of designs

In [333]:
designs_checked = trim_impossible_designs(tl2, designs)
print(len(designs_checked))

Processing 400 designs:
Processing design 0...
Processing design 100...
Processing design 200...
Processing design 300...
311


3. Get all solutions using the minimum pieces

In [334]:
min_solutions = get_possible_designs(tl2, designs_checked)

Processing 311 designs:
Processing design 0...
Processing design 100...
Processing design 200...
Processing design 300...


4. Get all solutions from the minimum piece solutions

In [335]:
#Get all valid combinations by taking the minimal solutions and finding all solutions where pieces can be combined
def get_valid_combos(pieces, cur_idx=0):
    global valid_histroy

    #Reset the history variable on first call
    if cur_idx == 0:
        valid_histroy = {}

    #If the remaining list combinations has already been calculated then simply return it
    if cur_idx in valid_histroy.keys():
        return valid_histroy[cur_idx]
    
    #If there are no more pieces to process then there are no more decisions to make
    if not pieces:
        valid_histroy[cur_idx] = 1 
        return 1
    
    #Count the number of combinations for each piece option at this index
    combinations = 0
    
    #Every option for the next character
    #Include picking no option and leaving the first piece as is
    cur_considerations = [(cur_idx, cur_idx)] + [x for x in pieces if x[0] == cur_idx]
    num_at_cur_idx = len(cur_considerations) - 1

    for piece in cur_considerations:
        #Get the length of the piece to set the next index
        piece_len = piece[1] - piece[0]
        #Get the number of combinations for the remainder of the pieces input at the next index
        combinations += get_valid_combos(pieces[num_at_cur_idx + piece_len:], cur_idx=cur_idx+piece_len+1)

    #After iterating all possibilities for this index, record the combinations in the history dict
    valid_histroy[cur_idx] = combinations
    return combinations

In [336]:
#Derrive combinations of solutions using the minimum solution as a basis
def derrive_combinations(min_solution, breakdown):
    combo_pieces = []

    #Get all combo pieces that can be inserted in the minimum solution
    for piece_idx in range(len(min_solution)-1):
        for length in range(piece_idx+2, len(min_solution)+1):
            combo = min_solution[piece_idx:length]
            if tuple(combo) in breakdown.keys():
                combo_pieces.append((piece_idx, piece_idx+len(combo)-1))

    #Determine valid combinations using the combo pieces
    return get_valid_combos(combo_pieces)

In [337]:
#Code to get output
combinations = 0
print("Processing", len(min_solutions), "designs:")

#Iterate for each design
for idx, design in enumerate(min_solutions):
    #Iterate again as a design might have multiple solutions using the minimum pieces
    for design_solution in design:
        #Get the number of solutions for this design
        res = derrive_combinations(design_solution, breakdown)
        combinations += res

    if idx % 30 == 0:
        print(idx, "Designs solved")

print(combinations)

Processing 311 designs:
0 Designs solved
30 Designs solved
60 Designs solved
90 Designs solved
120 Designs solved
150 Designs solved
180 Designs solved
210 Designs solved
240 Designs solved
270 Designs solved
300 Designs solved
616234236468263
